<h1 align="center">Computación de Alto Desempeño</h1>
<h1 align="center">Ejemplos de Programas Paralelos</h1>
<h1 align="center">2024</h1>
<h1 align="center">MEDELLÍN - COLOMBIA </h1>

*** 
|[![Outlook](https://img.shields.io/badge/Microsoft_Outlook-0078D4?style=plastic&logo=microsoft-outlook&logoColor=white)](mailto:calvarezh@udemedellin.edu.co)||[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/S09a_AlgoritmosParalelosConceptos.ipynb)
|-:|:-|--:|
|[![LinkedIn](https://img.shields.io/badge/linkedin-%230077B5.svg?style=plastic&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/carlosalvarez5/)|[![@alvarezhenao](https://img.shields.io/twitter/url/https/twitter.com/alvarezhenao.svg?style=social&label=Follow%20%40alvarezhenao)](https://twitter.com/alvarezhenao)|[![@carlosalvarezh](https://img.shields.io/badge/github-%23121011.svg?style=plastic&logo=github&logoColor=white)](https://github.com/carlosalvarezh)|

<table>
 <tr align=left><td><img align=left src="https://github.com/carlosalvarezh/Curso_CEC_EAFIT/blob/main/images/CCLogoColorPop1.gif?raw=true" width="25">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.(c) Carlos Alberto Alvarez Henao</td>
</table>

***

# Aviso Legal sobre estas Notas de Clase

Las presentes notas de clase han sido elaboradas con fines educativos y como material de apoyo para el aprendizaje y comprensión de la computación paralela. Estas notas están basadas en información y contenidos derivados del tutorial *["Introduction to Parallel Computing Tutorial"](https://hpc.llnl.gov/documentation/tutorials/introduction-parallel-computing-tutorial)* disponible en el sitio web del *Laboratorio Nacional Lawrence Livermore* (https://hpc.llnl.gov/). Este material se proporciona tal cual, sin garantías de exactitud completa o de la aplicabilidad para un fin particular.

A pesar de que se ha hecho un esfuerzo por asegurar la precisión y utilidad de estas notas, los usuarios deben tener en cuenta que los conceptos, aplicaciones y técnicas de la computación paralela están en constante evolución, y se recomienda consultar múltiples fuentes y la documentación oficial más actual para obtener la información más reciente y completa.

Por favor, considere que cualquier ejemplo, referencia o cita de "Introducción a la Computación Paralela" se proporciona con el objetivo de ilustrar los conceptos y principios básicos de la computación paralela y no debe considerarse como una guía exhaustiva o definitiva sobre el tema.
***

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel00.jpg?raw=true" width="500" />
</p>

## Ejemplos Paralelos

### Procesamiento de Arrays

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel57.gif?raw=true" width="150" />
</p>

Este ejemplo demuestra cálculos sobre elementos de un array bidimensional; se evalúa una función en cada elemento del array. La computación en cada elemento del array es independiente de otros elementos del array. El problema es intensivo computacionalmente.

El programa serial calcula un elemento a la vez en orden secuencial. El código serial podría ser de la forma:

```fortran
do j = 1, n
  do i = 1, n
    a(i,j) = fcn(i,j)
  end do
end do
```

***Preguntas a considerar:***
- ¿Este problema se puede paralelizar?
- ¿Cómo se podría particionar el problema?
- ¿Se necesitan comunicaciones?
- ¿Hay dependencias de datos?
- ¿Se necesitan sincronizaciones?
- ¿Será una preocupación el balance de carga?

***Solución Paralela 1: Procesamiento de Elementos de Array Distribuidos Uniformemente***

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel58.gif?raw=true" width="250" />
</p>

- El cálculo de los elementos es independiente uno de otro, lo que lleva a una solución paralela embarazosamente simple. 
- Los elementos del array están distribuidos uniformemente, de modo que cada proceso posee una porción del array (subarray).

  - El esquema de distribución se elige para un acceso eficiente a la memoria; por ejemplo, paso unitario (stride de 1) a través de los subarrays. El paso unitario maximiza el uso de la caché/memoria.

  - Dado que es deseable tener un paso unitario a través de los subarrays, la elección de un esquema de distribución depende del lenguaje de programación. Consulta el diagrama de Distribuciones Cíclicas por Bloques para ver las opciones.

- El cálculo independiente de los elementos del array asegura que no es necesaria la comunicación o sincronización entre tareas.

- Dado que la cantidad de trabajo está distribuida uniformemente entre los procesos, no deberían existir preocupaciones sobre el equilibrio de carga.

- Después de que el array es distribuido, cada tarea ejecuta la porción del bucle correspondiente a los datos que posee.

- Por ejemplo, se muestran tanto las distribuciones de bloques en Fortran (por columnas) como en C (por filas).


**Distribución por Columnas (Fortran)**:
```fortran
do j = mystart, myend 
  do i = 1, n 
    a(i,j) = fcn(i,j) 
  end do 
end do
```

**Distribución por Filas (C)**:
```c
for (int i = mystart; i < myend; i++) { 
  for (int j = 0; j < n; j++) { 
    a[i][j] = fcn(i, j); 
  }
}
```

Observa que solo las variables del bucle externo son diferentes de la solución serial. 

***Una posible solución:***
Implementar como un modelo de Programa Único Múltiples Datos (SPMD) - cada tarea ejecuta el mismo programa.
- **Proceso Maestro:** Inicializa el array, envía información a los procesos trabajadores y recibe los resultados.
- **Proceso Trabajador:** Recibe información, realiza su parte del cálculo y envía los resultados al maestro.
Usando el esquema de almacenamiento de Fortran, realiza una distribución por bloques del array.

***pseudocódigo:***

Pseudo código  paralelo: se resaltan en rojo los cambios para el paralelismo.

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel61.PNG?raw=true" width="350" />
</p>

***Solución Paralela 2: Grupo de Tareas***

La solución anterior de array demostró un balanceo de carga estático:
- Cada tarea tiene una cantidad fija de trabajo.
- Puede haber tiempo de inactividad significativo para procesadores más rápidos o menos cargados; las tareas más lentas determinan el rendimiento general.

Si tienes un problema de balanceo de carga (algunas tareas trabajan más rápido que otras), podrías beneficiarte de usar un esquema de "grupo de tareas".

**Esquema de Grupo de Tareas**
Dos procesos son empleados:
- **Proceso Maestro:** Mantiene un grupo de tareas para que los procesos trabajadores las realicen, envía una tarea cuando se solicita, recopila resultados de los trabajadores.
- **Proceso Trabajador:** Repetidamente hace lo siguiente:
  - Obtiene tarea del proceso maestro.
  - Realiza el cálculo.
  - Envía los resultados al maestro.

Los procesos trabajadores no saben antes de tiempo de ejecución qué parte del array manejarán o cuántas tareas realizarán. El balanceo de carga dinámico ocurre en tiempo de ejecución: las tareas más rápidas obtendrán más trabajo para hacer.

Pseudo código  paralelo: se resaltan en rojo los cambios para el paralelismo.

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel62.PNG?raw=true" width="350" />
</p>


En el ejemplo de grupo de tareas, cada tarea calculó un elemento individual del array como un trabajo. La relación de cálculo a comunicación es finamente granular. Las soluciones finamente granulares incurren en más sobrecarga de comunicación para reducir el tiempo de inactividad de las tareas. Una solución más óptima podría ser distribuir más trabajo con cada trabajo. La "cantidad correcta" de trabajo depende del problema.

### Cálculo de PI

#### Calcular PI de manera secuencial

El valor de PI puede calcularse de varias maneras. Considere el método de Monte Carlo para aproximar PI:
- Inscriba un círculo con radio $r$ en un cuadrado con longitud de lado de $2r$.
- El área del círculo es $\pi r^2$ y el área del cuadrado es $4r^2$.
- La proporción del área del círculo respecto al área del cuadrado es:
  $$\pi r^2 / 4r^2 = \pi / 4$$
- Si genera aleatoriamente $N$ puntos dentro del cuadrado, aproximadamente $N \pi / 4$ de esos puntos ($M$) deberían caer dentro del círculo.
- $\pi$ entonces se aproxima como:
$$N \pi / 4 = M$$
$$\pi / 4 = M / N$$
$$\pi = 4 M / N$$

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel59.gif?raw=true" width="250" />
</p>

Note que aumentar el número de puntos generados mejora la aproximación.

Pseudo código secuencial para este procedimiento:

```pseudocode
npoints = 10000
circle_count = 0

do j = 1,npoints
  generate 2 random numbers between 0 and 1
  xcoordinate = random1
  ycoordinate = random2
  if (xcoordinate, ycoordinate) inside circle
  then circle_count = circle_count + 1
end do

PI = 4.0*circle_count/npoints
```
El problema es computacionalmente intensivo—la mayoría del tiempo se gasta ejecutando el bucle.

***Preguntas para considerar:***
- ¿Es este problema paralelizable?
- ¿Cómo se dividiría el problema?
- ¿Se necesitan comunicaciones?
- ¿Hay dependencias de datos?
- ¿Se necesitan sincronizaciones?
- ¿Será una preocupación el balance de carga?

***Solución Paralela***

Otro problema que es fácil de paralelizar:
- Todos los cálculos de puntos son independientes; no hay dependencias de datos.
- El trabajo puede ser dividido equitativamente; no hay preocupaciones de balance de carga.
- No se necesita comunicación ni sincronización entre tareas.

Estrategia Paralela:
- Divida el bucle en partes iguales que puedan ser ejecutadas por el grupo de tareas.
- Cada tarea realiza de manera independiente su trabajo.
- Se utiliza un modelo SPMD.
- Una tarea actúa como maestro para recoger resultados y calcular el valor de PI.

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel60.gif?raw=true" width="250" />
</p>

Pseudo código  paralelo: se resaltan en rojo los cambios para el paralelismo.

<p float="center">
  <img src="https://github.com/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/images/Parallel63.PNG?raw=true" width="350" />
</p>